In [1]:
import matplotlib.pyplot as plt #Plotting
import numpy as np #Number manipulation
import re #String manipulation
import pandas as pd #Dataframe manipulation
import math #Math
import matplotlib #Plotting
#from __future__ import division #Math
import seaborn as sns #Better plotting
import scipy #Statistical tests
from keras.models import Sequential, Input, Model
from keras.layers import LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
%matplotlib inline 
from ipykernel import kernelapp as app


Using TensorFlow backend.


In [2]:
subjects = ["subject12study.csv", "subject13study.csv", "subject14study.csv", "subject15study.csv"]
infos = ["Subject 12.csv", "Subject 13.csv", "Subject 14.csv", "Subject 15.csv"]

data_main = []
data_side = []

for i in range(len(subjects)):
    df = pd.read_csv(subjects[i], delimiter = "\t")
    info = pd.read_csv(infos[i])

    # Cleaning the data
    df = df[df["avgx"] !=0] #eyex and eyey cannot be equal to 0
    df = df[df["avgy"] !=0]
    df = df[df["avgx"] > 0] #eyex should be bigger than 0 and smaller than 1920, regarding the screen size during the study
    df = df[df["avgx"] <1920]
    df = df[df["avgy"] > 0]  
    df = df[df["avgy"] < 1080] #eyey should be bigger than 0 and smaller than 1080, according to the screen size during the study
    df = df[df["psize"] < 25] #pupil size can be max 25 and min 13
    df = df[df["psize"] >13]

    data_main.append(df)
    data_side.append(info)


c:\users\gizem\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
datax = []

for q in range(len(data_main)):
    
    realtime = [] #realtime
    for a in data_main[q]["timestamp"]:
        try:
            d , f = a.split()
            (h, m, s) = f.split(':')
            result = float(h) * 3600 + float(m) * 60 + float(s)
            realtime.append(result)
        except:
            realtime.append(float(0))

    len(realtime)
    data_main[q]["realtime"] = realtime
    dfx = data_main[q][["realtime", "avgx", "avgy", "psize"]] #creating a new dataframe for only requiered columns
    
    datax.append(dfx)
    
    imgstart = []
    imgend =[]
    decstart =[]
    decenda = []
    decendb =[]


    for i in data_side[q]["ImgStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resulti = float(h)*3600 + float(m)*60 + float(s)
            imgstart.append(resulti)

    for i in data_side[q]["ImgEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = i.split(":")
            resultie = float(h)*3600 + float(m)*60 + float(s)
            imgend.append(resultie)

    for e in data_side[q]["EmptyStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultes = float(h)*3600 + float(m)*60 + float(s)
            decstart.append(resultes)

    for e in data_side[q]["EmptyEnd"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = e.split(":")
            resultee = float(h)*3600 + float(m)*60 + float(s)
            decenda.append(resultee)

    cro2beg = []
    contbeg = []
    cro1beg = []

    for c in data_side[q]["ComplexStart"]: #separating the stages of the cross1, context, cross2, images, decision depending on time
            (h,m,s) = c.split(":")
            result = float(h)*3600 + float(m)*60 + float(s)
            decendb.append(result)
            cro2beg.append(result-9)
            contbeg.append(result-13)
            cro1beg.append(result-14)

    #print (imgstart[1], imgend[1], decstart[1], decenda[1], decendb[1] )

    data_side[q]["Cro1Beg"] = cro1beg  #cross1(fixation)
    data_side[q]["ContBeg"] = contbeg  #context
    data_side[q]["Cro2Beg"] = cro2beg  #cross2(fixation)
    data_side[q]["ImagesBeg"] = imgstart    #images stage started
    data_side[q]["ImagesEnd"] = imgend
    data_side[q]["DecisionBeg"] = decstart   #decision stage started  (green frame around the pictures)
    data_side[q]["DecisionEnd"] = decenda


    data_side[q].sort_values("DecisionEnd", inplace=True, ascending=True) #reorganizing dataframe

    data_side[q] = data_side[q].reset_index(drop=True) #saving df

In [4]:
"""
datac[0][4][79]
for i in range(80):
    print(len(datac[0][4][i]))
"""

'\ndatac[0][4][79]\nfor i in range(80):\n    print(len(datac[0][4][i]))\n'

In [5]:
#len(datax), datax[0].shape, len(data_side), data_side[0].shape
#len(datac), len(datac[0]), len(datac[0][0])
#len(datac), len(datac[0]), len(datac[0][0]), len(datac[0][0][1])

In [6]:

datac = []

for q in range(len(datax)):
    
    cross1 = []
    context = []
    cross2 = []
    images = []
    decision=[]
    sub = []
    
    for i in range(80):
        cross1.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro1Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ContBeg"][i]))])
        context.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ContBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["Cro2Beg"][i]))])
        cross2.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["Cro2Beg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesBeg"][i]))])
        images.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["ImagesBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["ImagesEnd"][i]))])
        decision.append(datax[q].loc[((datax[q]['realtime']) >= float(data_side[q]["DecisionBeg"][i])) & ((datax[q]["realtime"]) <= float(data_side[q]["DecisionEnd"][i]))])
    
    sub.append(cross1)
    sub.append(context)
    sub.append(cross2)
    sub.append(images)
    sub.append(decision)
    
    datac.append(sub)

In [7]:
Cx = [] #choice data as left and right
for i in range(4):
    c = []
    c = data_side[i]['Choice'].tolist()
    Cx.append(c)

In [8]:
len(datac[0][4]), len(Cx[0])

xdata = []
cdata = []


for i in range(4): # only image state
    for q in range(80):
        xdata.append(datac[i][3][q])
        cdata.append(Cx[i][q])
        
print(len(xdata), len(cdata))
print (xdata[1])


320 320
       realtime       avgx      avgy     psize
2967  45358.518   952.6624  515.9188  16.11940
2968  45358.552   955.2593  513.2438  16.16895
2969  45358.585   958.1833  511.9755  16.29105
2970  45358.618   957.6060  512.9666  16.11715
2971  45358.652   958.1423  514.7060  15.60410
2972  45358.685   957.3959  514.6401  15.64320
2973  45358.718   957.5872  514.3387  15.70275
2974  45358.751  1364.6797  491.5477  15.43870
2975  45358.785  1540.9421  515.9224  15.47570
2976  45358.818  1531.4805  521.6324  15.34300
2977  45358.852  1532.8319  514.7095  15.36290
2978  45358.885  1534.5177  513.9880  15.65700
2979  45358.918  1538.0160  515.2425  15.20790
2980  45358.951  1537.0378  513.0066  15.25225
2981  45358.985  1563.0913  515.4323  15.26590
2982  45359.018  1531.2725  509.2605  15.31415
2983  45359.051  1533.4116  507.2269  15.62820
2984  45359.085  1547.7240  508.0655  15.45175
2985  45359.118  1556.6954  508.5528  15.57855
2986  45359.151  1565.4314  509.7057  15.20870
2987 

In [9]:
nnx = []
nnc = []

for i in range(len(xdata)):
    #print(len(xdata[i]), cdata[i])
    
    if len(xdata[i]) >= 80:
        a = xdata[i][-80:] #Gets last 80 data points, use [:100] to get the first 100
        z = a.values[:,1:]
        print(z)
        nnx.append(z)
        nnc.append(cdata[i])
    else:
        pass


#print(len(nnx), len(nnx[46]), len(nnc))
#print(z.shape, z)



[[ 379.405    447.4371    15.9666 ]
 [ 379.3479   448.2762    15.89735]
 [ 376.3811   449.0978    15.68775]
 [ 375.8947   446.5077    15.90315]
 [ 377.3763   449.0862    16.3014 ]
 [ 380.131    449.7231    16.177  ]
 [ 381.6593   449.8635    16.2806 ]
 [ 382.2018   449.9324    16.2127 ]
 [ 381.29     449.8145    16.16585]
 [ 380.9102   450.8437    16.23845]
 [ 380.2889   451.7495    16.1883 ]
 [ 380.5038   452.0938    16.36155]
 [ 380.6509   452.5644    16.5023 ]
 [ 381.5641   452.4608    16.4067 ]
 [ 382.5909   452.3818    16.5732 ]
 [ 381.9304   452.3909    16.55485]
 [ 384.9817   452.4924    16.82735]
 [1021.7537   405.917     17.04365]
 [1451.7239   455.0707    16.7925 ]
 [1440.271    456.187     16.93065]
 [1442.1083   457.3056    16.8096 ]
 [1442.6235   462.2432    16.8391 ]
 [1440.8607   462.7127    16.80015]
 [1436.406    452.0146    16.88955]
 [1439.8024   462.9093    16.77985]
 [1438.5325   464.4228    16.86925]
 [1433.4507   462.8611    16.5549 ]
 [1435.9075   466.8721    16

 [941.7734  511.9225   15.73595]]
[[ 432.9386   536.5859    13.7261 ]
 [ 426.3942   558.6741    14.1167 ]
 [ 421.0339   567.6514    13.85125]
 [ 415.9366   573.2777    13.9218 ]
 [ 415.0413   582.1292    13.7854 ]
 [ 408.2334   588.8026    13.96615]
 [ 406.716    592.5972    13.9737 ]
 [ 405.5374   596.4375    14.055  ]
 [ 405.6264   600.2872    13.8101 ]
 [ 406.9369   602.9421    13.70995]
 [ 405.6343   606.7997    13.9368 ]
 [ 405.0277   611.9354    13.9265 ]
 [ 406.3324   615.481     14.1078 ]
 [ 405.7905   617.8339    14.2767 ]
 [ 865.3765   582.4345    14.1846 ]
 [1370.938    550.4569    14.0809 ]
 [1424.7798   556.8315    14.62925]
 [1438.9193   563.1121    14.69795]
 [1454.8168   564.8923    14.31645]
 [1464.085    568.3618    14.45215]
 [1491.0243   589.5531    14.3646 ]
 [1463.7307   550.5125    14.4346 ]
 [1499.4283   542.8184    15.0379 ]
 [1518.7345   549.3785    14.8577 ]
 [1527.1133   545.4286    14.81945]
 [1533.7585   542.114     15.05575]
 [1536.2987   544.1545    14.7

[[ 399.3143   478.4618    15.1697 ]
 [ 399.5988   479.6113    15.1729 ]
 [ 400.0755   479.0747    15.19865]
 [ 401.8254   477.8535    15.2573 ]
 [ 401.6121   476.7999    14.9938 ]
 [ 401.5373   476.1011    15.06615]
 [ 401.7789   474.6072    15.13985]
 [ 401.3423   473.3016    15.121  ]
 [ 400.8759   474.8777    15.45835]
 [ 761.9625   456.8336    14.94325]
 [1399.178    408.7118    15.98905]
 [1501.8286   487.9103    13.5717 ]
 [1539.6757   472.7517    15.67145]
 [1544.5021   476.9952    15.3243 ]
 [1547.9784   475.9407    15.0023 ]
 [1545.4778   478.8219    14.8275 ]
 [1562.9983   514.7049    14.9802 ]
 [1561.3295   493.2473    14.84615]
 [1563.1705   491.1511    14.95465]
 [1560.3055   489.3831    14.9008 ]
 [1561.1046   491.6424    14.88105]
 [1569.8409   492.5452    14.71495]
 [1568.645    495.3579    14.82225]
 [1566.9846   495.6102    14.6839 ]
 [1571.7939   495.0404    14.47255]
 [1575.516    493.9548    14.6448 ]
 [1578.9838   494.5967    14.3581 ]
 [1581.936    497.8177    14

 [1593.818    457.6297    14.12215]]
[[ 337.7009   479.7946    14.26025]
 [ 336.6982   481.1247    14.43695]
 [ 348.5959   482.2053    14.5649 ]
 [1117.7753   495.3054    14.3036 ]
 [1603.3867   448.14      14.81095]
 [1610.1991   449.4013    14.7623 ]
 [1608.792    450.1705    15.03485]
 [1610.1891   454.9907    14.82095]
 [1611.1987   461.2075    15.08745]
 [1588.5354   469.7712    14.91795]
 [1570.2418   461.9875    14.98425]
 [1568.2598   472.7597    14.9285 ]
 [1562.2985   471.613     15.0879 ]
 [1560.7369   477.5805    15.0534 ]
 [1559.6017   478.3846    13.78605]
 [1559.037    480.1982    13.82515]
 [1562.299    480.5895    13.7716 ]
 [1563.5403   479.3855    13.6707 ]
 [1565.8389   477.8363    13.7553 ]
 [1577.974    489.0429    13.09265]
 [1579.4471   486.1348    13.9651 ]
 [1580.1075   483.1599    13.62245]
 [1582.8005   482.709     14.00545]
 [1582.3499   482.7127    13.5833 ]
 [1582.4056   480.1215    13.5189 ]
 [1581.798    477.3176    13.81245]
 [1582.4894   474.829     1

 [ 359.9785   431.961     13.9974 ]]
[[ 385.9774   490.5923    13.61055]
 [ 377.8758   487.9099    14.0159 ]
 [ 374.5555   487.5229    13.61055]
 [ 373.1179   488.7422    13.10755]
 [ 372.0484   490.5688    13.50035]
 [ 369.8193   491.2284    13.30955]
 [ 368.3548   492.1764    13.3156 ]
 [ 369.4495   494.3671    13.2286 ]
 [ 369.6957   494.4721    13.32295]
 [ 369.3046   493.7485    13.8001 ]
 [ 368.5363   492.8644    13.46685]
 [ 367.3165   492.4514    13.6258 ]
 [ 368.6822   492.8251    13.62705]
 [ 367.8994   493.9236    13.4654 ]
 [ 367.8481   493.999     13.5745 ]
 [ 367.9527   494.1245    13.34545]
 [ 370.0652   496.6149    13.58545]
 [ 373.4208   496.9457    13.4294 ]
 [ 378.5574   496.3932    13.569  ]
 [ 384.2873   498.0587    13.48125]
 [ 387.6323   499.5598    13.47545]
 [ 390.1482   499.4106    13.3089 ]
 [ 395.1428   499.93      13.7456 ]
 [ 399.6623   502.3208    13.7764 ]
 [ 728.9665   475.9854    14.928  ]
 [1401.808    475.4262    13.2076 ]
 [1447.6976   476.3199    1

 [1555.2019   504.2505    16.9008 ]]
[[1504.024    526.0657    17.6608 ]
 [1507.2363   523.6161    17.3131 ]
 [1510.4618   522.7268    17.3642 ]
 [1511.2212   521.3392    17.287  ]
 [1503.765    520.5759    17.34035]
 [ 702.3561   448.1678    16.9019 ]
 [ 406.4473   497.6311    17.3835 ]
 [ 400.5653   493.5316    17.14495]
 [ 395.8461   494.2217    16.8573 ]
 [ 395.3051   491.7687    16.83575]
 [ 393.5458   492.171     16.66275]
 [ 385.7958   497.2344    16.71775]
 [ 374.144    494.0873    16.60915]
 [ 381.3872   493.9287    16.7901 ]
 [ 383.6306   494.9094    16.66005]
 [ 384.8559   496.0665    16.7716 ]
 [ 384.3004   499.2198    16.94165]
 [ 384.5152   498.5829    16.68245]
 [ 381.6862   496.4944    16.9253 ]
 [ 382.1357   496.3996    16.7543 ]
 [ 383.7845   495.2242    16.94205]
 [ 385.4037   496.3955    16.79005]
 [ 386.131    498.5327    17.0139 ]
 [ 385.6273   497.3752    16.84015]
 [ 387.5793   497.4896    16.88765]
 [1040.6902   456.6435    16.9017 ]
 [1521.6932   512.2682    1

 [1554.2422   504.0054    16.08795]]
[[1554.8408   551.5687    18.62045]
 [1557.3506   548.8879    18.12285]
 [1558.7869   545.4691    17.9788 ]
 [1558.4626   540.4048    16.4675 ]
 [1546.9243   537.3969    16.39685]
 [ 715.7552   491.6459    15.89115]
 [ 410.0319   528.2996    16.03095]
 [ 410.2607   532.2881    15.90635]
 [ 410.1187   536.7732    15.9293 ]
 [ 407.7488   532.5667    15.89475]
 [ 407.7915   534.2224    15.74155]
 [ 384.8357   531.7415    15.342  ]
 [ 398.4573   535.7932    15.47155]
 [ 396.1105   530.2798    15.96975]
 [ 394.6013   533.0062    15.7294 ]
 [ 393.991    532.8364    15.6448 ]
 [ 395.7805   532.2236    15.9314 ]
 [ 396.6075   530.8787    15.87205]
 [ 397.9912   529.6571    15.71765]
 [ 396.4528   528.9428    15.83835]
 [ 395.3112   528.9456    15.6382 ]
 [ 394.0624   528.3249    15.9607 ]
 [ 393.5211   528.5614    15.80875]
 [ 394.7276   529.118     16.11705]
 [ 394.4201   528.627     15.79675]
 [ 396.0064   529.2932    16.13325]
 [ 396.5356   528.7365    1

 [ 381.5881   480.1148    16.421  ]]
[[1559.2902   448.1678    16.5556 ]
 [1556.9556   449.0466    16.4823 ]
 [1557.5144   449.6042    16.9088 ]
 [1556.8781   449.5604    17.22015]
 [1556.8241   459.9452    16.9418 ]
 [1557.1794   470.5499    16.8331 ]
 [1559.2657   477.6552    16.92075]
 [1558.1259   483.7674    17.01225]
 [1558.094    489.8365    16.843  ]
 [1557.3796   495.7897    17.03555]
 [1555.3199   499.808     16.91845]
 [1551.7959   502.0269    17.1444 ]
 [1548.3815   504.7059    16.22355]
 [1545.9247   507.0886    16.40835]
 [1544.0552   509.0899    16.229  ]
 [1541.8533   508.0362    16.0102 ]
 [1540.2505   506.8779    16.0312 ]
 [1538.2136   507.2464    16.18525]
 [1536.0923   505.3167    15.77145]
 [1532.724    503.3384    15.8754 ]
 [1530.1052   502.2871    16.0655 ]
 [ 723.269    442.0702    16.1073 ]
 [ 357.9771   496.8834    16.83125]
 [ 377.6533   497.335     17.1835 ]
 [ 390.213    494.0287    17.098  ]
 [ 395.4934   495.9613    16.84575]
 [ 397.4669   498.3304    1

 [1531.6464   503.793     15.80825]]
[[1556.171    530.7283    15.7699 ]
 [1557.0718   531.7871    16.1322 ]
 [1558.7384   531.6985    16.19115]
 [ 880.4664   492.7804    15.7284 ]
 [ 326.9594   508.1041    15.36595]
 [ 338.5532   519.3446    15.47035]
 [ 354.544    515.1297    15.84105]
 [ 352.4668   515.1748    15.4972 ]
 [ 351.7383   518.2037    15.7169 ]
 [ 344.2904   512.8679    15.71385]
 [ 347.565    504.027     15.59225]
 [ 342.688    522.7857    15.67555]
 [ 345.9089   519.163     15.62875]
 [ 335.787    522.9016    15.5714 ]
 [ 331.0184   518.9563    15.4836 ]
 [ 327.2931   511.3067    15.16745]
 [ 325.5037   511.3693    15.0721 ]
 [ 321.4447   511.5995    15.7123 ]
 [ 321.2786   513.4276    15.437  ]
 [ 321.7939   512.4476    15.3577 ]
 [ 317.8742   510.904     15.38325]
 [ 314.7636   512.0888    15.28995]
 [ 311.1144   511.5569    15.3084 ]
 [ 595.5371   483.0616    15.2251 ]
 [1356.5881   461.2907    15.25355]
 [1445.3431   469.0399    15.47285]
 [1484.5872   474.3143    1

 [1525.4292   577.726     15.3652 ]]
[[1498.6323   556.5837    15.0634 ]
 [1499.0525   557.0404    15.08345]
 [1498.5502   558.1214    15.0803 ]
 [1498.4358   557.3115    15.0416 ]
 [1498.5571   556.4404    14.8854 ]
 [1499.441    555.947     15.40925]
 [1499.7128   555.8707    15.24555]
 [1500.4241   555.7389    15.1533 ]
 [1501.7148   557.0589    15.2689 ]
 [1502.0005   556.5884    15.53655]
 [1502.8848   556.8279    15.60045]
 [1503.0118   557.4847    15.6795 ]
 [1502.3027   557.8486    15.58085]
 [1500.4453   558.0175    15.7246 ]
 [1500.1176   558.6703    15.77795]
 [1497.1453   558.5505    15.61945]
 [1494.7894   558.6702    15.54215]
 [1491.4026   558.3249    15.8343 ]
 [1258.4204   517.8721    15.2753 ]
 [ 554.7529   473.4896    15.27215]
 [ 461.6166   477.0898    15.4431 ]
 [ 434.3246   486.2069    15.68875]
 [ 425.9766   489.6721    15.6137 ]
 [ 417.8521   486.9309    15.59315]
 [ 375.874    523.4591    15.75635]
 [ 400.1482   525.012     15.56215]
 [ 379.1722   510.1922    1

In [10]:
for i in nnx:
    i[:,0] = i[:,0]/1920
    i[:,1] = i[:,1]/1080
    i[:,2] = (i[:,2]-13)/12

In [11]:
import random
sayko = list(zip(nnx,nnc)) 
random.shuffle(sayko)
nnx, nnc = zip(*sayko)
valx = nnx[:30]
valy = nnc[:30]

In [12]:
nn_x = np.stack(nnx[30:])
nn_c = np.stack(nnc[30:])
val_x = np.stack(valx)
val_y = np.stack(valy)
print(nn_x.shape, nn_c.shape, val_x.shape, val_y.shape)


(282, 80, 3) (282,) (30, 80, 3) (30,)


In [13]:
d= []
for i in nn_c:
    if i == 1:
        k = [1,0]
        d.append(k)
    elif i == 0:
        k = [0,1]
        d.append(k)

e = np.stack(d).reshape(282,2,1)
f = nn_x.reshape(282,240)
g = val_x.reshape(30,240)
f.shape

(282, 240)

In [20]:
inputs = Input(shape=(240,))

x = Dense(16, activation='relu')(inputs)
x = Dense(8, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=prediction)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(f, nn_c, epochs=150, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 240)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                3856      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 4,001
Trainable params: 4,001
Non-trainable params: 0
_________________________________________________________________
Train on 225 samples, validate on 57 samples
Epoch 1/150
 - 1s - loss: 0.6691 - binary_accuracy: 0.5911 - val_loss: 0.6817 - val_binary_accuracy: 0.5965
Epoch 2/150
 - 0s - loss: 0.6488 - binary_accuracy: 0.6356 - val_loss: 0.6614 - val_binary_accuracy: 0.6316
Epoch 3/150

Epoch 68/150
 - 0s - loss: 0.5195 - binary_accuracy: 0.7289 - val_loss: 0.5190 - val_binary_accuracy: 0.7368
Epoch 69/150
 - 0s - loss: 0.5351 - binary_accuracy: 0.7289 - val_loss: 0.5207 - val_binary_accuracy: 0.7368
Epoch 70/150
 - 0s - loss: 0.5202 - binary_accuracy: 0.7467 - val_loss: 0.5203 - val_binary_accuracy: 0.7544
Epoch 71/150
 - 0s - loss: 0.5163 - binary_accuracy: 0.7333 - val_loss: 0.5202 - val_binary_accuracy: 0.7544
Epoch 72/150
 - 0s - loss: 0.5116 - binary_accuracy: 0.7467 - val_loss: 0.5209 - val_binary_accuracy: 0.7544
Epoch 73/150
 - 0s - loss: 0.5235 - binary_accuracy: 0.7333 - val_loss: 0.5249 - val_binary_accuracy: 0.7544
Epoch 74/150
 - 0s - loss: 0.5134 - binary_accuracy: 0.7467 - val_loss: 0.5178 - val_binary_accuracy: 0.7368
Epoch 75/150
 - 0s - loss: 0.5110 - binary_accuracy: 0.7467 - val_loss: 0.5210 - val_binary_accuracy: 0.7544
Epoch 76/150
 - 0s - loss: 0.5121 - binary_accuracy: 0.7333 - val_loss: 0.5362 - val_binary_accuracy: 0.7544
Epoch 77/150
 - 0s 

Epoch 143/150
 - 0s - loss: 0.4698 - binary_accuracy: 0.7911 - val_loss: 0.5262 - val_binary_accuracy: 0.7544
Epoch 144/150
 - 0s - loss: 0.4727 - binary_accuracy: 0.7644 - val_loss: 0.5476 - val_binary_accuracy: 0.7544
Epoch 145/150
 - 0s - loss: 0.4769 - binary_accuracy: 0.7822 - val_loss: 0.5238 - val_binary_accuracy: 0.7544
Epoch 146/150
 - 0s - loss: 0.4608 - binary_accuracy: 0.7867 - val_loss: 0.5117 - val_binary_accuracy: 0.7895
Epoch 147/150
 - 0s - loss: 0.4657 - binary_accuracy: 0.7689 - val_loss: 0.5437 - val_binary_accuracy: 0.7018
Epoch 148/150
 - 0s - loss: 0.4639 - binary_accuracy: 0.7733 - val_loss: 0.5476 - val_binary_accuracy: 0.7368
Epoch 149/150
 - 0s - loss: 0.4749 - binary_accuracy: 0.7644 - val_loss: 0.5166 - val_binary_accuracy: 0.7719
Epoch 150/150
 - 0s - loss: 0.4654 - binary_accuracy: 0.7644 - val_loss: 0.5085 - val_binary_accuracy: 0.7719


In [15]:
inputs = Input(shape=(240,))

x = Dense(8, activation='relu')(inputs)
x = Dense(4, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)


model2 = Model(inputs=inputs, outputs=prediction)
model2.summary()
model2.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])
history = model2.fit(f, nn_c, epochs=150, shuffle= True, batch_size=64, verbose=2, validation_split=0.2 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 240)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 1928      
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 1,969
Trainable params: 1,969
Non-trainable params: 0
_________________________________________________________________
Train on 225 samples, validate on 57 samples
Epoch 1/150
 - 1s - loss: 0.2592 - binary_accuracy: 0.5289 - val_loss: 0.2544 - val_binary_accuracy: 0.5088
Epoch 2/150
 - 0s - loss: 0.2502 - binary_accuracy: 0.5289 - val_loss: 0.2468 - val_binary_accuracy: 0.5088
Epoch 3/150

Epoch 68/150
 - 0s - loss: 0.1823 - binary_accuracy: 0.7422 - val_loss: 0.1752 - val_binary_accuracy: 0.7018
Epoch 69/150
 - 0s - loss: 0.1827 - binary_accuracy: 0.7378 - val_loss: 0.1746 - val_binary_accuracy: 0.7018
Epoch 70/150
 - 0s - loss: 0.1827 - binary_accuracy: 0.7244 - val_loss: 0.1755 - val_binary_accuracy: 0.7544
Epoch 71/150
 - 0s - loss: 0.1818 - binary_accuracy: 0.7244 - val_loss: 0.1747 - val_binary_accuracy: 0.7544
Epoch 72/150
 - 0s - loss: 0.1808 - binary_accuracy: 0.7200 - val_loss: 0.1738 - val_binary_accuracy: 0.7193
Epoch 73/150
 - 0s - loss: 0.1803 - binary_accuracy: 0.7289 - val_loss: 0.1734 - val_binary_accuracy: 0.7193
Epoch 74/150
 - 0s - loss: 0.1803 - binary_accuracy: 0.7289 - val_loss: 0.1731 - val_binary_accuracy: 0.7368
Epoch 75/150
 - 0s - loss: 0.1810 - binary_accuracy: 0.7289 - val_loss: 0.1728 - val_binary_accuracy: 0.7193
Epoch 76/150
 - 0s - loss: 0.1801 - binary_accuracy: 0.7200 - val_loss: 0.1731 - val_binary_accuracy: 0.7544
Epoch 77/150
 - 0s 

Epoch 143/150
 - 0s - loss: 0.1689 - binary_accuracy: 0.7378 - val_loss: 0.1689 - val_binary_accuracy: 0.7544
Epoch 144/150
 - 0s - loss: 0.1688 - binary_accuracy: 0.7333 - val_loss: 0.1688 - val_binary_accuracy: 0.7544
Epoch 145/150
 - 0s - loss: 0.1693 - binary_accuracy: 0.7333 - val_loss: 0.1689 - val_binary_accuracy: 0.7544
Epoch 146/150
 - 0s - loss: 0.1689 - binary_accuracy: 0.7289 - val_loss: 0.1687 - val_binary_accuracy: 0.7544
Epoch 147/150
 - 0s - loss: 0.1688 - binary_accuracy: 0.7289 - val_loss: 0.1689 - val_binary_accuracy: 0.7544
Epoch 148/150
 - 0s - loss: 0.1707 - binary_accuracy: 0.7244 - val_loss: 0.1699 - val_binary_accuracy: 0.7719
Epoch 149/150
 - 0s - loss: 0.1678 - binary_accuracy: 0.7289 - val_loss: 0.1688 - val_binary_accuracy: 0.7544
Epoch 150/150
 - 0s - loss: 0.1684 - binary_accuracy: 0.7244 - val_loss: 0.1687 - val_binary_accuracy: 0.7719


In [21]:
s = model.predict(g)

In [22]:
model.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 1ms/step


[0.6316641569137573, 0.699999988079071]

In [18]:
model2.evaluate(x=g, y=val_y, verbose=1, sample_weight=None, steps=1000)

1000/1000 [==============================] - 1s 835us/step


[0.21909162402153015, 0.6333333253860474]

In [19]:
for i in range(30):
    if s[i] > 0.5:
        print(1, val_y[i])
    else:
        print(0, val_y[i])

0 1.0
0 0.0
1 1.0
0 1.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
1 1.0
1 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
1 0.0
0 0.0
1 1.0
1 1.0
0 1.0
0 1.0
0 0.0
1 1.0
0 0.0
1 0.0
0 1.0
1 0.0
1 1.0
